In [249]:
import numpy as np
from time import time
import heapq


# Pierres isotropic is optimal

In [242]:
def to_binary_array(i, n):
    # a binary array of length n representing i. For example if i=5 and n=5 this would return [0, 0, 1, 0, 1].
    array = [int(d) for d in str(bin(i))[2:]]
    if len(array) < n:
        extra_zeros = [0 for _ in range(n-len(array))]
        return extra_zeros + array
    return array


def set_and_sort(l):
    # returns the unique elemenets of l in an ordered list (np.isclose is used to determine if elements are equal)
    l = sorted(list(set(l)))
    new_l = []
    for i in range(len(l)-1):
        if np.isclose(l[i], l[i+1]):
            pass
        else:
            new_l.append(l[i])
    new_l.append(l[-1])
    return new_l

In [251]:
for n in range(9): # length of the input 
    print(n)
    start = time()
    for x_int in range(2**n): # the input graph
        for delta_int in range(2**n): # the perturbation 
            x = np.array(to_binary_array(x_int, n))
            delta = np.array(to_binary_array(delta_int, n))

            # all possible zs in arrays
            zs = []
            for i in range(2**n):
                zs.append(to_binary_array(i, n))
            zs = np.array(zs)

            # ratios calculated using Eq. 9 in my document (decomposed into 4 products)
            p = 0.05
            ratios = []
            for z in zs:
                ratio = 1.0
                for i in range(len(z)):
                    if delta[i] == 1:
                        if x[i] == z[i]:
                            ratio = ratio * p/(1-p)
                        else:
                            ratio = ratio * (1-p)/p
                ratios.append(ratio)
            ratios = set_and_sort(ratios)

            L = sum(delta==1) # size of the perturbation

            # wang ratios 
            wang = []
            for i in range(-L, L+1):
                wang.append((p/(1-p))**i)
            wang = set_and_sort(wang)

            # pierre ratios
            L = sum(delta==1)
            pierre = []
            for i in range(-L, L+1, 2):
                pierre.append((p/(1-p))**i)
            pierre = set_and_sort(pierre)

            assert np.allclose(pierre, ratios) # pierres ratios are exactly the observed ratios 
            
    print(n, time() - start)

0
0 0.00028133392333984375
1
1 0.0007824897766113281
2
2 0.004415988922119141
3
3 0.024610280990600586
4
4 0.13481473922729492
5
5 0.7258450984954834
6
6 4.396568059921265
7
7 33.83141779899597
8
8 225.94684743881226


# Sorting the alphas

In [259]:
p = 0.01, 0.021
sols = {}
for i in range(10):
    for j in range(10):
        if i + j > 10:
            continue
        else:
            key = (i, j)
            vals = p[0]*i + p[1]*j
            sols[key] = vals
sols = sort_dict(sols)
sols

{(0, 0): 0.0,
 (1, 0): 0.01,
 (2, 0): 0.02,
 (0, 1): 0.021,
 (3, 0): 0.03,
 (1, 1): 0.031,
 (4, 0): 0.04,
 (2, 1): 0.041,
 (0, 2): 0.042,
 (5, 0): 0.05,
 (3, 1): 0.051000000000000004,
 (1, 2): 0.052000000000000005,
 (6, 0): 0.06,
 (4, 1): 0.061,
 (2, 2): 0.062,
 (0, 3): 0.063,
 (7, 0): 0.07,
 (5, 1): 0.07100000000000001,
 (3, 2): 0.07200000000000001,
 (1, 3): 0.073,
 (8, 0): 0.08,
 (6, 1): 0.081,
 (4, 2): 0.082,
 (2, 3): 0.083,
 (0, 4): 0.084,
 (9, 0): 0.09,
 (7, 1): 0.09100000000000001,
 (5, 2): 0.092,
 (3, 3): 0.093,
 (1, 4): 0.094,
 (8, 1): 0.101,
 (6, 2): 0.10200000000000001,
 (4, 3): 0.10300000000000001,
 (2, 4): 0.10400000000000001,
 (0, 5): 0.10500000000000001,
 (9, 1): 0.111,
 (7, 2): 0.11200000000000002,
 (5, 3): 0.113,
 (3, 4): 0.114,
 (1, 5): 0.115,
 (8, 2): 0.122,
 (6, 3): 0.123,
 (4, 4): 0.124,
 (2, 5): 0.125,
 (0, 6): 0.126,
 (7, 3): 0.133,
 (5, 4): 0.134,
 (3, 5): 0.135,
 (1, 6): 0.136,
 (6, 4): 0.14400000000000002,
 (4, 5): 0.14500000000000002,
 (2, 6): 0.146,
 (0, 7): 

In [252]:
p = (0.01, 0.015, 0.03, 0.07)

In [253]:
sols = {}
for i in range(10):
    for j in range(10):
        for k in range(10):
            for l in range(10):
                if i + j + l + k > 10:
                    continue
                else:
                    key = (i, j, k, l)
                    val = p[0]*i + p[1]*j + p[2]*k + p[3]*l
                    sols[key] = val

In [255]:
def sort_dict(x):
    return {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}

In [319]:
sols = sort_dict(sols)
sols
len(sols)

64

In [264]:
d = 4
p = (0.01, 0.015, 0.03, 0.07)

In [315]:
heap = [(0, tuple([0 for _ in range(d)]))] # create heap
seen = set((0, 0, 0, 0))
for i in range(10):
    # retrieve best item
    next_item = heapq.heappop(heap)
    next_item_tuple = next_item[1]

    # generate new candidates
    for j in range(d):
        pass
        candidate = next_item_tuple[:j] + (next_item_tuple[j]+1,) + next_item_tuple[j+1:]
        if candidate not in seen:
            value = np.inner(candidate, p)
            heapq.heappush(heap, (value, candidate))
            seen.add(candidate)

In [316]:
def yield_likelihood_ratios(p):
    d = len(p)
    heap = [(0, tuple([0 for _ in range(d)]))] # create heap
    seen = set((0, 0, 0, 0))
    while True:
        # retrieve best item
        next_item = heapq.heappop(heap)
        next_item_tuple = next_item[1]
        yield next_item

        # generate new candidates
        for j in range(d):
            pass
            candidate = next_item_tuple[:j] + (next_item_tuple[j]+1,) + next_item_tuple[j+1:]
            if candidate not in seen:
                value = np.inner(candidate, p)
                heapq.heappush(heap, (value, candidate))
                seen.add(candidate)

In [318]:
total_p = 0.0
for region in yield_likelihood_ratios(p):
    total_p += region[0]
    print(region)
    if total_p > 0.5:
        break

(0, (0, 0, 0, 0))
(0.01, (1, 0, 0, 0))
(0.015, (0, 1, 0, 0))
(0.02, (2, 0, 0, 0))
(0.025, (1, 1, 0, 0))
(0.03, (0, 0, 1, 0))
(0.03, (0, 2, 0, 0))
(0.03, (3, 0, 0, 0))
(0.035, (2, 1, 0, 0))
(0.04, (1, 0, 1, 0))
(0.04, (1, 2, 0, 0))
(0.04, (4, 0, 0, 0))
(0.045, (0, 1, 1, 0))
(0.045, (0, 3, 0, 0))
(0.045, (3, 1, 0, 0))
(0.05, (2, 0, 1, 0))
(0.05, (2, 2, 0, 0))
